In [8]:
import gym
import torch
from myrl.buffers import ReplayBuffer, PrioritizedReplayBuffer
from myrl.visualizer import showit
from myrl.utils import ExperimentWriter, get_batch_obs, get_batch_a, Discrete2Continuous
from myrl.value_functions import polyak, DQN
from myrl.policies import RandomPolicy
from myrl.environments import Envs


wll = ExperimentWriter('tb/dqn_hhhh')
envname = 'BipedalWalker-v3'
# envname = 'LunarLander-v2'
# envname = 'CartPole-v0'
# envname = 'MountainCar-v0'
envname = 'Pendulum-v0'
# envname = 'Boxing-ram-v0'
# envname = 'Breakout-ram-v0'
# envname = 'Acrobot-v1'
# envname = 'FetchPickAndPlace-v1'

env = gym.make(envname)
envs = Envs(envname, 1)
env.observation_space.sample()

array([-0.8188496,  0.7931332, -5.7149386], dtype=float32)

In [9]:
rbuff = ReplayBuffer(10000)

In [16]:
print(env.action_space, env.observation_space)
# adim = env.action_space.n 
adim = 7
sdim = env.observation_space.shape[0]
dqn = DQN([sdim, 512, 256, adim])
dqn_cont = Discrete2Continuous(-2, 2, adim, dqn.act).act
import copy
tdqn = copy.deepcopy(dqn)
dqn

Box(1,) Box(3,)


DQN(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=7, bias=True)
  )
)

In [17]:
opt = torch.optim.Adam(dqn.parameters(), lr=1e-1)

In [20]:
wll.new()
writer = wll.writer
bsize = 128
warmup = 100
gamma = 0.97

collector = lambda obs: dqn.act(obs, epsilon=0.4)
cont_collector = Discrete2Continuous(-2, 2, adim, collector).act
shower    = lambda obs: dqn.act(obs, epsilon=0)
random_policy = RandomPolicy(env).act


for ep in range(0, 100000):
    if ep%1==0:
        pi = cont_collector if len(rbuff)>bsize*50 else random_policy
        oldobs, _, r, obs, d, _, a, _ = envs.rollout(pi, debug=0)
        rbuff.add(oldobs, a, r, obs, d)
    writer.add_scalar('dqn/reward', r.mean(), ep)
    rew = r.sum().item()
    # print(rew)

    if bsize*5 > len(rbuff):
        print(len(rbuff), end='\r')
        continue

    for jup in range(20):
        oldobs, a, r, obs, done = rbuff.get(bsize)
        for opt_step in range(3):
            target = tdqn.get_max(obs) * (1-done)
            calc = dqn.get_q(oldobs, a)
            loss = ((r + gamma*target - calc)**2).mean()
            opt.zero_grad()
            loss.backward()
            opt.step()          
        polyak(dqn, tdqn, 1-1/100)


    writer.add_scalar('dqn/model', loss.item(), ep)

    
    if ep%10==0:
        print(ep, loss.item(), rew, "jup")
    if ep%30==0:
        print("showit = ", showit(env, collector, time_delta=0.01, max_steps=1000))

0 4.182734489440918 -1336.53662109375 jup
showit =  -1625.1324291191117
10 2.9133734703063965 -1307.2371826171875 jup
20 7.75437593460083 -1190.545654296875 jup
30 4.599780082702637 -1591.1768798828125 jup
showit =  -1484.1492859219982
40 13.360116958618164 -1539.18359375 jup
50 83.07841491699219 -1336.1104736328125 jup
60 7.462909698486328 -1242.3614501953125 jup
showit =  -1371.034450926845
70 8.520977973937988 -1526.0106201171875 jup
80 267.107177734375 -1342.6588134765625 jup
90 126.30908203125 -1145.4527587890625 jup
showit =  -1685.5130157065503
100 200.96038818359375 -1039.0223388671875 jup
110 9.251940727233887 -1571.8043212890625 jup
120 175.14283752441406 -1337.908203125 jup
showit =  -1430.8112466527878
130 25.10015296936035 -1307.2589111328125 jup
140 106.80274963378906 -1058.7205810546875 jup
150 19.402557373046875 -1105.2777099609375 jup
showit =  -1277.055446032577
160 8.977272987365723 -1293.6563720703125 jup
170 368.5322265625 -1091.4091796875 jup
180 16.46692276000976

KeyboardInterrupt: 

In [14]:
# novishower = lambda obs: dqn.act(obs, 0.0)
showit(env, cont_collector ,time_delta=0.01)

198 /2000

-1496.324793678495